In [1]:
import bempp.api, numpy as np
from math import pi
import os

from Grid_Maker import *

mol_name = 'arg' ; stern_thickness = 0 ; probe_radius = 1.4
mesh_density = 10.0 ; min_area = 0 

ep_m = 4.
ep_s = 80.
k = 0.125

e_c = 1.60217662e-19 # [C] - proton charge
k_B = 1.38064852e-23 # [m2 kg s-2 K-1]
T   = 298. # [K]  
C = 1. # e_c/(k_B*T)

pqr_to_xyzr(mol_name , stern_thickness , method = 'amber' )
xyzr_to_msh(mol_name , mesh_density , probe_radius , stern_thickness , min_area , Mallador = 'MSMS')

path = 'Molecule/'+mol_name
triangle_areas( path , mol_name , mesh_density )

bempp.api.set_ipython_notebook_viewer()
bempp.api.PLOT_BACKEND = "ipython_notebook"


path = 'Molecule/'+mol_name+'/'

grid_name_File =  os.path.join(path,mol_name + '_'+str(mesh_density)+'.msh')

grid = bempp.api.import_grid(grid_name_File)


q, x_q = np.empty(0), np.empty((0,3))

pqr_file = os.path.join(path,mol_name+'.pqr')
charges_file = open( pqr_file , 'r').read().split('\n')

for line in charges_file:
    line = line.split()
    if len(line)==0: continue
    if line[0]!='ATOM': continue
    q = np.append( q, float(line[8]))
    x_q = np.vstack( ( x_q, np.array(line[5:8]).astype(float) ) )  
    
from bempp.api.operators.boundary import sparse, laplace, modified_helmholtz

dirichl_space = bempp.api.function_space(grid,  "DP", 0)
neumann_space = bempp.api.function_space(grid,  "DP", 0) 
dual_to_dir_s = bempp.api.function_space(grid,  "DP", 0)

identity = sparse.identity(     dirichl_space, dirichl_space, dual_to_dir_s)
slp   = laplace.single_layer(neumann_space, dirichl_space, dual_to_dir_s)
dlp   = laplace.double_layer(dirichl_space, dirichl_space, dual_to_dir_s)

def zero_i(x, n, domain_index, result):
    result[:] = 0
    
def u_s_G(x,n,domain_index,result):
    global q,x_q,ep_m,C
    result[:] = C / (4.*np.pi*ep_m)  * np.sum( q / np.linalg.norm( x - x_q, axis=1 ) )
    
def du_s_G(x,n,domain_index,result):
    global q,x_q,ep_m,C
    result[:] = -C/(4.*np.pi*ep_m)  * np.sum( np.dot( x-
                            x_q , n)  * q / np.linalg.norm( x - x_q, axis=1 )**3 )
    
u_s  = bempp.api.GridFunction(dirichl_space, fun=u_s_G)
du_s = bempp.api.GridFunction(neumann_space, fun=du_s_G)

from bempp.api.operators.boundary import sparse, laplace, modified_helmholtz

identity = sparse.identity(     dirichl_space, dirichl_space, dual_to_dir_s)
slp_in   = laplace.single_layer(neumann_space, dirichl_space, dual_to_dir_s)
dlp_in   = laplace.double_layer(dirichl_space, dirichl_space, dual_to_dir_s)

k_p = k # kappa'

identity = sparse.identity(     dirichl_space, dirichl_space, dual_to_dir_s)
slp_in   = laplace.single_layer(neumann_space, dirichl_space, dual_to_dir_s)
dlp_in   = laplace.double_layer(dirichl_space, dirichl_space, dual_to_dir_s)

slp_out  = modified_helmholtz.single_layer(neumann_space, dirichl_space, dual_to_dir_s, k_p)
dlp_out  = modified_helmholtz.double_layer(dirichl_space, dirichl_space, dual_to_dir_s, k_p)

blocked = bempp.api.BlockedOperator(2, 2)
blocked[0, 0] = 0.5*identity + dlp_in
blocked[0, 1] = -slp_in
blocked[1, 0] = 0.5*identity - dlp_out
blocked[1, 1] = (ep_m/ep_s)*slp_out


zero = bempp.api.GridFunction(dirichl_space, fun=zero_i)

rhs = [ zero ,  -slp_out *(ep_m/ep_s)*du_s]
sol, info,it_count = bempp.api.linalg.gmres( blocked , rhs, return_iteration_count=True, tol=1e-4)
print("The linear system was solved in {0} iterations".format(it_count))
u_r , du_r = sol
print('Regular component solved.')

.xyzr File from .pqr ready.
Normal .vert & .face Done
Normal .msh Done
Mesh Ready
The linear system was solved in 120 iterations
Regular component solved.


In [2]:
u_tot = u_r + u_s
du_tot= du_r + du_s

# Se definen las integrales de superficie, válido para espacios DP0 unicamente.
def Integral_superficie( A , B ):
    A = A.real.coefficients
    B = B.real.coefficients
    Area_F = open(path+'triangleAreas_'+str(mesh_density)+'.txt').readlines()
    Areas  = np.zeros(len(Area_F))
    for i in range(len(Area_F)):
        Areas[i] = float(Area_F[i][:-1]) 
    return A*B*Areas

# Se abre el archivo de texto que guarda las áreas por elemento
area_txt = open('Molecule/{0:s}/triangleAreas_{1:s}.txt'.format(mol_name , str(mesh_density) ) ).read().split('\n')
areas    = [float(i) for i in area_txt[:-1]]

In [5]:
ep_m*np.sum(Integral_superficie( u_tot , du_r) - Integral_superficie(du_tot , u_r))

0.0011537979622319849

In [4]:
from bempp.api.operators.potential import laplace as lp

slp_in_O = lp.single_layer(neumann_space, x_q.transpose()) 
dlp_in_O = lp.double_layer(dirichl_space, x_q.transpose())

phi_r_i = slp_in_O * du_r  -  dlp_in_O * u_r

S_phi     = np.sum(q * phi_r_i).real
print(S_phi)

0.00115412685676


In [6]:
intento_de_variable = Integral_superficie( u_tot , du_r) - Integral_superficie(du_tot , u_r)

In [8]:
intento_de_variable

array([  2.81783130e-08,   4.60308794e-08,   4.27616677e-08, ...,
         3.24856901e-07,   5.65675315e-07,   5.77215447e-07])

In [ ]:
ax.plot_trisurf(x, y, z, linewidth=0.2, antialiased=True)